In [14]:
from pathlib import Path

import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import seaborn as sns
from plotly.subplots import make_subplots

from yellowbrick.regressor import ResidualsPlot
import os
import dask.dataframe as dd #pour importer rapidement la base de données
import sys

In [33]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# Importation de la base

In [19]:
project_root = Path(os.getcwd()).parent  # ou spécifiez directement le chemin
sys.path.append(str(project_root))

# Construire le chemin relatif au projet
file_path = project_root / "bases" / "data2_bon1.csv"

In [23]:
data = pd.read_csv(file_path)

# Nettoyage du dataset

Le nettoyage du dataset consiste en ces principaux points:  


*   Supprimer(filtrer) les valeurs abberrantes(nulles, négatives et trop élévées ) de fare_amount:

le tarif ne doit pas être négatif, il ne doit pas être nul non plus et dans le cadre de New-York, le tarif le plus cher ne dépasse pas 150 $

*  supprimer les valeurs nulles et supérieures à 6 de passenger_count: le nombre de passagers d'un taxi à  NYC ne dépasse pas 6

* Supprimer les latitudes et longitudes hors_norme

In [29]:
def filter(df):
    # Apply all filtering conditions sequentially
    filtered_df = df[
        df["pickup_longitude"].between(-180, 180) &
        df["dropoff_longitude"].between(-180, 180) &
        df["pickup_latitude"].between(-90, 90) &
        df["dropoff_latitude"].between(-90, 90)
    ]
    return filtered_df

# Apply the filter function
data2 = filter(data)
data2= data2[data2.fare_amount > 0]
data2= data2[data2.passenger_count > 0]

# La mise en place des modeles

## Modele de regression simple

### La mise en place du pipeline pour le modele

In [44]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), ['hour_cos','passenger_count', 'distance','pickup_longitude','dropoff_longitude','pickup_latitude','dropoff_latitude']),
        
    ]
)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression()) 
])

### L'entrainement du modele 

In [47]:
pipeline.fit(X_train, y_train)

NameError: name 'X_train' is not defined

### Nous allons donner un nom a notre modele

In [ ]:
model = pipeline.named_steps['model']

## Modele de regression Ridge

### Creation du Pipeline

In [ ]:
pipeline = Pipeline([
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), variables_numeriques)  # Normaliser les variables numériques
        ], 
        remainder='passthrough'  # Laisser les autres colonnes intactes
    )),
    ('ridge', Ridge())  # Modèle Ridge
])

### Les alphas sur lequel chercher le meileur

In [56]:
alpha_values = [18.5,18.4,18,45]

### le mise en place du modele avec 

In [ ]:
param_grid = {'ridge__alpha': alpha_values}  # Utilisation de l'étape 'ridge' du pipeline
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring=make_scorer(r2_score))

### Chercher le meilleur apha

In [ ]:
best_alpha = grid_search.best_params_['ridge__alpha']
print(f"Meilleure valeur pour alpha : {best_alpha}")

### Entrainement le modele avec le meileur alpha

In [ ]:
best_model = grid_search.best_estimator_